In [16]:
import requests
import csv
from bs4 import BeautifulSoup

base_url = 'https://www.amazon.in'
search_url = 'https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_{}'

num_pages = 20  # Number of pages to scrape
products = []

for page in range(1, num_pages + 1):
    url = search_url.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    search_results = soup.find_all('div', {'data-component-type': 's-search-result'})

    for result in search_results:
        product_url = base_url + result.find('a', {'class': 'a-link-normal s-no-outline'}).get('href')
        product_name = result.find('span', {'class': 'a-size-medium a-color-base a-text-normal'}).text.strip()

        price_element = result.find('span', {'class': 'a-offscreen'})
        product_price = price_element.text if price_element else 'N/A'

        rating_element = result.find('span', {'class': 'a-icon-alt'})
        rating = rating_element.text.split()[0] if rating_element else 'N/A'

        reviews_element = result.find('span', {'class': 'a-size-base'})
        num_reviews = reviews_element.text.replace(',', '') if reviews_element else 'N/A'

        product_info = {
            'Product URL': product_url,
            'Product Name': product_name,
            'Product Price': product_price,
            'Rating': rating,
            'Number of Reviews': num_reviews
        }

        products.append(product_info)

# Scrape additional information for each product
for product in products:
    url = product['Product URL']
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    description_element = soup.find('div', {'id': 'productDescription'})
    description = description_element.text.strip() if description_element else 'N/A'

    asin_element = soup.find('th', text='ASIN')
    asin = asin_element.find_next_sibling('td').text.strip() if asin_element else 'N/A'

    product_description_element = soup.find('div', {'id': 'productDescription_feature_div'})
    product_description = product_description_element.text.strip() if product_description_element else 'N/A'

    manufacturer_element = soup.find('th', text='Manufacturer')
    manufacturer = manufacturer_element.find_next_sibling('td').text.strip() if manufacturer_element else 'N/A'

    product['Description'] = description
    product['ASIN'] = asin
    product['Product Description'] = product_description
    product['Manufacturer'] = manufacturer

# Save the scraped product information to a CSV file
filename = 'amazon_products.csv'
fieldnames = ['Product URL', 'Product Name', 'Product Price', 'Rating', 'Number of Reviews', 'Description', 'ASIN',
              'Product Description', 'Manufacturer']

with open(filename, 'w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(products)

print('Data saved to', filename)


Data saved to amazon_products.csv
